OK, just want to test out reading in the galaxy images from AWS

In [16]:
import astropy.io.ascii as ascii
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import astropy.visualization as viz
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import astropy.wcs as WCS # you need this astropy module for processing fits files
import matplotlib as mpl
import numpy as np
import tqdm
from astropy.table import Table, join
% inline matplotlib

In [2]:
filename = 'galaxyzoo2--assets--14-07-2017.csv'
path = '/Users/jegpeek/Dropbox/xorastro/'

In [3]:
data = ascii.read(path + filename, format='csv', fast_reader=False)

In [4]:
response = requests.get(data[90]['location'])
img = Image.open(BytesIO(response.content))

In [5]:
red, green, blue = img.split()

In [6]:
imgcube = np.zeros([424, 424, len(data)])

In [7]:
for i, d in enumerate(data[0:10]):
    response = requests.get(d['location'])
    img = Image.open(BytesIO(response.content))
    red, green, blue = img.split()
    imgcube[:, :, i] = green # confusingly green is SDSS r band

In [9]:
imgcube[:, :, 0]

array([[  0.,   0.,   5., ...,   3.,   4.,   5.],
       [  0.,   1.,   3., ...,   5.,   5.,   6.],
       [  3.,   2.,   2., ...,   5.,   4.,   4.],
       ..., 
       [  6.,   7.,   7., ...,   4.,   5.,  10.],
       [  4.,   7.,   9., ...,   6.,   5.,   8.],
       [  4.,   9.,  13., ...,   7.,   4.,   6.]])

In [14]:
len(data)

355990

Now we are going to read in the metadata header to cross-match on ID

In [17]:
metadata = Table.read(path + "xorastro_metadata_dr7id.fits")

In [18]:
len(metadata)

664393

In [29]:
data.rename_column('name', 'dr7objid')

In [19]:
metadata

objid,ra,dec,u,g,r,i,z,dr7objid
int64,float64,float64,float32,float32,float32,float32,float32,int64
1237648702968627397,188.358919127,-1.13789162121,19.9896,18.3103,17.7158,17.3921,17.1477,587722981743722704
1237648702985994607,228.063297529,-1.08079048805,18.8233,16.9535,16.0956,15.6736,15.3461,587729778523963718
1237648703515656271,211.566820575,-0.641101471062,18.8287,16.8623,15.9147,15.4594,15.134,587722982290751578
1237648703515721922,211.693298897,-0.788083074138,19.4108,17.4369,16.4775,16.0366,15.6531,587722982290817229
1237648703515787525,211.870049173,-0.7409002147,20.2416,18.285,17.3289,16.9156,16.5698,587722982290882809
1237648703515787559,211.904639367,-0.76476649291,18.3414,17.424,17.0828,16.7945,16.6918,587722982290882831
1237648703515852909,211.907136082,-0.806731147013,19.9837,18.2474,17.4737,17.0693,16.7534,587722982290948228
1237648703515918459,212.080249294,-0.648259519948,19.3668,17.56,16.7218,16.3067,15.9879,587722982291013783
1237648703515918471,212.085592618,-0.675436604757,19.5534,18.0383,17.2769,16.8942,16.6295,587722982291013790


In [32]:
joined = join(data, metadata, keys=['dr7objid'])

In [35]:
len(joined)

247590

In [36]:
joined

id,dr7objid,project_id,location,classification_count,external_ref,created_at,updated_at,region,redshift_bin,magsize_bin,battle_bin,ra_1,dec_1,stripe82,stripe82_coadd,extra_original,objid,ra_2,dec_2,u,g,r,i,z
int64,int64,int64,str45,int64,str83,str19,str19,int64,int64,int64,int64,float64,float64,int64,int64,int64,int64,float64,float64,float32,float32,float32,float32,float32
245610,587722981736054938,1,http://s3.amazonaws.com/zoo2_extra/245610.jpg,45,http://cas.sdss.org/dr7/en/tools/quicklook/quickobj.asp?id=587722981736054938,2009-09-02 06:31:02,2010-04-12 21:12:00,2,-9999,0,-9999,170.80557936,-1.18214647,0,0,1,1237674648850923796,170.805581176,-1.18213971519,19.7683,17.7893,16.9265,16.4148,16.0305
3,587722981741363294,1,http://s3.amazonaws.com/zoo2/3.jpg,63,http://cas.sdss.org/dr7/en/tools/quicklook/quickobj.asp?id=587722981741363294,2009-01-30 15:52:07,2010-01-15 02:32:13,2,6,66,66006,182.925262451,-1.09235703945,0,0,0,1237674648856232075,182.925269483,-1.09236649386,19.3509,17.3391,16.4046,15.9533,15.648
4,587722981741363323,1,http://s3.amazonaws.com/zoo2/4.jpg,45,http://cas.sdss.org/dr7/en/tools/quicklook/quickobj.asp?id=587722981741363323,2009-01-30 15:52:07,2010-03-30 23:16:57,2,5,77,77005,182.970108032,-1.21953725815,0,0,0,1237674648856232112,182.970126477,-1.21953915091,18.5501,16.6486,15.6884,15.2109,14.902
245612,587722981741428877,1,http://s3.amazonaws.com/zoo2_extra/245612.jpg,52,http://cas.sdss.org/dr7/en/tools/quicklook/quickobj.asp?id=587722981741428877,2009-09-02 06:31:02,2010-04-14 21:01:06,2,10,34,350,183.02862215,-1.07270517,0,0,1,1237648702966333581,183.028626368,-1.07271147736,19.1197,17.3641,16.7064,16.2256,16.0127
5,587722981741559888,1,http://s3.amazonaws.com/zoo2/5.jpg,53,http://cas.sdss.org/dr7/en/tools/quicklook/quickobj.asp?id=587722981741559888,2009-01-30 15:52:07,2009-12-31 02:14:02,2,999,0,999,183.438095093,-1.23841392994,0,0,0,1237648702966464577,183.438108663,-1.23841758168,20.5083,18.7491,17.3561,16.8188,16.5199
6,587722981741625481,1,http://s3.amazonaws.com/zoo2/6.jpg,58,http://cas.sdss.org/dr7/en/tools/quicklook/quickobj.asp?id=587722981741625481,2009-01-30 15:52:07,2010-03-13 03:54:58,2,7,95,95007,183.473999023,-1.23142921925,0,0,0,1237648702966530213,183.473985817,-1.23142963416,17.6526,16.4827,15.864,15.4917,15.2874
7,587722981741625484,1,http://s3.amazonaws.com/zoo2/7.jpg,52,http://cas.sdss.org/dr7/en/tools/quicklook/quickobj.asp?id=587722981741625484,2009-01-30 15:52:07,2010-01-18 21:32:06,2,9,25,25009,183.477783203,-1.08460354805,0,0,0,1237648702966530219,183.477780976,-1.08459747567,19.8283,17.6795,16.6919,16.2476,15.9
8,587722981741625520,1,http://s3.amazonaws.com/zoo2/8.jpg,45,http://cas.sdss.org/dr7/en/tools/quicklook/quickobj.asp?id=587722981741625520,2009-01-30 15:52:07,2010-04-08 07:30:10,2,6,24,24006,183.526199341,-1.16401135921,0,0,0,1237648702966530252,183.526202347,-1.16399295952,19.0773,17.1875,16.2938,15.875,15.5688
9,587722981741625545,1,http://s3.amazonaws.com/zoo2/9.jpg,46,http://cas.sdss.org/dr7/en/tools/quicklook/quickobj.asp?id=587722981741625545,2009-01-30 15:52:07,2010-02-25 20:46:58,2,8,46,46008,183.566818237,-1.13589453697,0,0,0,1237648702966530271,183.566819734,-1.13588105522,19.3423,17.4014,16.3897,15.9234,15.5544
